In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install pyarrow


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import zipfile 
import io
import os
import concurrent.futures
import time
import subprocess
import multiprocessing
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow
import json
from matplotlib.ticker import FormatStrFormatter

In [5]:
import sys
import os

sys.path.append(os.path.abspath('../advmla_at3_package'))

In [6]:
df_train = pd.read_feather('../data/processed/train_data.feather')
df_test = pd.read_feather('../data/processed/test_data.feather')

In [7]:
df_train.shape

(5287832, 15)

In [8]:
df_test.shape

(1321959, 15)

In [9]:
df_train.head(10)

,searchDate,flightDate,startingAirport,destinationAirport,isNonStop,isRefundable,isBasicEconomy,totalFare,totalTravelDistance,segmentsArrivalAirportCode,DepartureTimeHour,CabinCode,AirlineNameScore,date_diff_days,weekday
6164974,2022-04-18,2022-05-10,SFO,DFW,True,False,False,283.600006,1468.0,DFW,13,1.0,4,22,1
63647,2022-04-21,2022-05-10,ATL,MIA,True,False,False,35.970001,596.0,MIA,10,1.0,1,19,1
5067940,2022-04-17,2022-06-05,OAK,ATL,False,False,False,334.579987,NaN,LAX||ATL,6,1.0,1,49,6
3385047,2022-04-28,2022-06-27,LAX,BOS,False,False,False,353.600006,2666.0,PHX||BOS,18,1.0,4,60,0
2982409,2022-04-19,2022-04-28,JFK,ATL,True,False,True,148.600006,762.0,ATL,12,1.0,4,9,3
4039661,2022-05-18,2022-07-01,LGA,ORD,True,False,False,327.600006,720.0,ORD,14,1.0,2,44,4
781738,2022-04-18,2022-05-05,BOS,DFW,False,False,False,152.600006,1565.0,JFK||DFW,12,1.0,2,17,3
3644365,2022-04-17,2022-05-18,LAX,ORD,False,False,False,417.600006,1787.0,PHX||ORD,13,1.0,4,31,2
2765255,2022-05-05,2022-05-18,JFK,BOS,False,False,False,497.200012,627.0,IAD||BOS,14,1.0,4,13,2
3107360,2022-04-25,2022-05-20,JFK,BOS,False,False,False,487.200012,627.0,IAD||BOS,14,1.0,4,25,4


In [24]:
df_train.isnull().sum()

searchDate                         0
flightDate                         0
startingAirport                    0
destinationAirport                 0
isNonStop                          0
isRefundable                       0
isBasicEconomy                     0
totalFare                          0
totalTravelDistance           484828
segmentsArrivalAirportCode         0
DepartureTimeHour                  0
CabinCode                          0
AirlineNameScore                   0
date_diff_days                     0
weekday                            0
dtype: int64

In [10]:
df_test.isnull().sum()

searchDate                         0
flightDate                         0
startingAirport                    0
destinationAirport                 0
isNonStop                          0
totalFare                          0
totalTravelDistance           120986
segmentsArrivalAirportCode         0
DepartureTimeHour                  0
CabinCode                          0
AirlineNameScore                   0
date_diff_days                     0
weekday                            0
dtype: int64

In [10]:
from engineering.clean_master import preprocessing_data

df_train_processed, df_test_precessed = preprocessing_data(df_train, df_test)

In [11]:
selected_features = ['date_diff_days', 'weekday', 'CabinCode', 'DepartureTimeHour', 'totalTravelDistance', 'isRefundable', 'isBasicEconomy', 'isNonStop']
categorical_features = ['startingAirport', 'destinationAirport']
target = 'totalFare'

df_filtered_train = df_train_processed[categorical_features + selected_features + [target]]
df_filtered_test = df_test_precessed[categorical_features + selected_features + [target]]

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LinearRegression

numerical_features = ['totalTravelDistance']

preprocessor = ColumnTransformer(
    transformers=[
        ('one-hot', OneHotEncoder(handle_unknown='ignore'), categorical_features),  
        ('scaler', StandardScaler(), numerical_features) 
    ],
    remainder='passthrough')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression()) 
])

In [13]:
from sklearn.model_selection import train_test_split


# Split the datasets
train_df, val_df = train_test_split(df_filtered_train, test_size=0.2, random_state=42)

X_train = train_df.drop(columns=[target], axis=1)  
y_train = train_df[target]

X_val = val_df.drop(columns=[target], axis=1)  
y_val = val_df[target]

X_test = df_filtered_test.drop(columns=[target], axis=1)  
y_test = df_filtered_test[target]

In [14]:
from modeling.null import NullRegressor

base_model = NullRegressor()

y_base = base_model.fit_predict(y_train)


In [15]:
# Baseline result

from modeling.performance import print_regressor_scores

print_regressor_scores(y_preds=y_base, y_actuals=y_train, set_name='Training')
print_regressor_scores(y_preds=base_model.predict(y_val), y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_preds=base_model.predict(y_test), y_actuals=y_test, set_name='Testing')

RMSE Training: 189.79791259765625
MAE Training: 136.0172119140625
RMSE Validation: 189.61782836914062
MAE Validation: 135.9205780029297
RMSE Testing: 188.92431640625
MAE Testing: 135.76583862304688


In [16]:
#1. Linear Regression

pipeline.fit(X_train, y_train)

y_preds = pipeline.predict(X_train)
y_val_preds = pipeline.predict(X_val)
y_test_preds = pipeline.predict(X_test)

In [17]:
#1. Linear Regression result

print_regressor_scores(y_preds, y_train, set_name='Training')
print_regressor_scores(y_val_preds, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds, y_actuals=y_test, set_name='Testing')

RMSE Training: 131.97960557816188
MAE Training: 92.59253783978536
RMSE Validation: 131.72065132793418
MAE Validation: 92.47262614164494
RMSE Testing: 131.56070187574915
MAE Testing: 92.54072417509839


In [30]:
import joblib
model_path = 'linear_model.joblib'
joblib.dump(pipeline, model_path)

['linear_model.joblib']

In [39]:
from sklearn.linear_model import SGDRegressor

pipeline3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('SGD', SGDRegressor(learning_rate='optimal',alpha=0.1)) 
])

pipeline3.fit(X_train, y_train)

y_preds3 = pipeline3.predict(X_train)
y_val_preds3 = pipeline3.predict(X_val)
y_test_preds3 = pipeline3.predict(X_test)

In [41]:
#2. SGDRegressor result

print_regressor_scores(y_preds3, y_train, set_name='Training')
print_regressor_scores(y_val_preds3, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds3, y_actuals=y_test, set_name='Testing')

RMSE Training: 663988798.303233
MAE Training: 527912128.62180895
RMSE Validation: 663994738.175438
MAE Validation: 528064419.2527296
RMSE Testing: 664383560.1067221
MAE Testing: 528123134.92055064


In [42]:
pipeline_sgd = Pipeline([
    ('preprocessor', preprocessor),
    ('SGD', SGDRegressor(
        learning_rate='invscaling',  
        eta0=0.001,  
        max_iter=1000,  
        tol=1e-4  
    ))
])

pipeline_sgd.fit(X_train, y_train)

y_preds3 = pipeline_sgd.predict(X_train)
y_val_preds3 = pipeline_sgd.predict(X_val)
y_test_preds3 = pipeline_sgd.predict(X_test)

In [43]:
#3. SGDRegressor result (try different learning rate)

print_regressor_scores(y_preds3, y_train, set_name='Training')
print_regressor_scores(y_val_preds3, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds3, y_actuals=y_test, set_name='Testing')

RMSE Training: 136.7813937479851
MAE Training: 96.24451014140384
RMSE Validation: 135.9198738981014
MAE Validation: 96.18253896091979
RMSE Testing: 136.51112215924726
MAE Testing: 96.27562736429101


In [25]:
from sklearn.ensemble import AdaBoostRegressor

pipeline4 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('Ada', AdaBoostRegressor()) 
])

pipeline4.fit(X_train, y_train)

y_preds3 = pipeline4.predict(X_train)
y_val_preds3 = pipeline4.predict(X_val)
y_test_preds3 = pipeline4.predict(X_test)

In [26]:
#4. Adaboost result

print_regressor_scores(y_preds3, y_train, set_name='Training')
print_regressor_scores(y_val_preds3, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds3, y_actuals=y_test, set_name='Testing')

RMSE Training: 239.69478899890268
MAE Training: 192.6699526841691
RMSE Validation: 239.48726029444512
MAE Validation: 192.60243057179738
RMSE Testing: 239.75430269125368
MAE Testing: 192.68703559240902


In [31]:
from sklearn.ensemble import RandomForestRegressor

pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('random_forest', RandomForestRegressor(
        n_estimators=20,
        max_depth=5,
        max_features='log2',
        random_state=42
    ))
])

pipeline_rf.fit(X_train, y_train)

y_preds5 = pipeline_rf.predict(X_train)
y_val_preds5 = pipeline_rf.predict(X_val)
y_test_preds5 = pipeline_rf.predict(X_test)

In [34]:
#5. Randomforest result

print_regressor_scores(y_preds5, y_train, set_name='Training')
print_regressor_scores(y_val_preds5, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds5, y_actuals=y_test, set_name='Testing')

RMSE Training: 153.66209223155462
MAE Training: 110.71380431801786
RMSE Validation: 152.62209640108296
MAE Validation: 110.6031855614612
RMSE Testing: 153.5224195293117
MAE Testing: 110.76359140367751


In [35]:
from sklearn.ensemble import RandomForestRegressor

pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('random_forest', RandomForestRegressor(
        n_estimators=20,
        max_depth=5,
        max_features='sqrt',
        random_state=42
    ))
])

pipeline_rf.fit(X_train, y_train)

y_preds5 = pipeline_rf.predict(X_train)
y_val_preds5 = pipeline_rf.predict(X_val)
y_test_preds5 = pipeline_rf.predict(X_test)

In [36]:
#5. Randomforest result (try different max_features)

print_regressor_scores(y_preds5, y_train, set_name='Training')
print_regressor_scores(y_val_preds5, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds5, y_actuals=y_test, set_name='Testing')

RMSE Training: 148.66311192808527
MAE Training: 106.95499720718799
RMSE Validation: 147.63094347906735
MAE Validation: 106.86490898985168
RMSE Testing: 148.62350695127662
MAE Testing: 107.0206381875522


In [37]:
from sklearn.ensemble import RandomForestRegressor

pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('random_forest', RandomForestRegressor(
        n_estimators=50,
        max_depth=5,
        max_features='sqrt',
        random_state=42
    ))
])

pipeline_rf.fit(X_train, y_train)

y_preds5 = pipeline_rf.predict(X_train)
y_val_preds5 = pipeline_rf.predict(X_val)
y_test_preds5 = pipeline_rf.predict(X_test)

In [38]:
#5. Randomforest result (try different n_estimators)

print_regressor_scores(y_preds5, y_train, set_name='Training')
print_regressor_scores(y_val_preds5, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds5, y_actuals=y_test, set_name='Testing')

RMSE Training: 147.3470493157371
MAE Training: 105.14035042981885
RMSE Validation: 146.3135251874094
MAE Validation: 105.06231659574148
RMSE Testing: 147.31635763754997
MAE Testing: 105.20337972359265


## Choose linear regression as a best model, optimise its performance

In [44]:
from sklearn.linear_model import ElasticNet

elastic_model = ElasticNet(alpha=1.0, l1_ratio=0.5)

pipeline_ela = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('elastic', ElasticNet(alpha=1.0, l1_ratio=0.5)) 
])

pipeline_ela.fit(X_train, y_train)

y_preds6 = pipeline_ela.predict(X_train)
y_val_preds6 = pipeline_ela.predict(X_val)
y_test_preds6 = pipeline_ela.predict(X_test)

In [45]:
#1. ElasticNet result

print_regressor_scores(y_preds6, y_train, set_name='Training')
print_regressor_scores(y_val_preds6, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds6, y_actuals=y_test, set_name='Testing')

RMSE Training: 161.54868516178252
MAE Training: 108.85429637746299
RMSE Validation: 160.2458962104936
MAE Validation: 108.76579881143603
RMSE Testing: 161.8102993976421
MAE Testing: 108.94793961291643


In [32]:
# Apply PolynomialFeatures

from sklearn.preprocessing import PolynomialFeatures

pipeline_poly = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('polynomial_features', PolynomialFeatures(degree=2)),  # Add polynomial features
    ('regressor', LinearRegression())
])


pipeline_poly.fit(X_train, y_train)

y_preds6 = pipeline_poly.predict(X_train)
y_val_preds6 = pipeline_poly.predict(X_val)
y_test_preds6 = pipeline_poly.predict(X_test)

In [33]:
#2. Linear Regression model after adding PolynomialFeatures (Best model)

print_regressor_scores(y_preds6, y_train, set_name='Training')
print_regressor_scores(y_val_preds6, y_actuals=y_val, set_name='Validation')
print_regressor_scores(y_test_preds6, y_actuals=y_test, set_name='Testing')

RMSE Training: 122.27469787101582
MAE Training: 84.72047258335992
RMSE Validation: 122.14766860068697
MAE Validation: 84.61828455194961
RMSE Testing: 121.99713262875125
MAE Testing: 84.71366449363327


In [34]:
predictions_df = pd.DataFrame(y_test_preds6, columns=['lr_predicted_totalFare'])
predictions_df.to_csv('../data/processed/predictions_lr.csv', index=False)

In [ ]:
import joblib
model_path = 'linear_model_final.joblib'
joblib.dump(pipeline, model_path)